In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 9.5 MB/s eta 0:00:00


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.6 MB/s eta 0:00:00


In [13]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score,
                             confusion_matrix, roc_auc_score, matthews_corrcoef)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import KNNImputer
import optuna
from sklearn.model_selection import cross_val_score

In [5]:

# Load the dataset
df = pd.read_csv('kepler.csv', comment='#')

df.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,...,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,0,...,0.200,0.160,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,Done,2018-08-16,CANDIDATE,0.969,0,0,...,0.000,0.480,0.390,0.360,0.490,0.340,0.120,0.730,0.500,0.450
2,10811496,K00753.01,NaN,CANDIDATE,Done,2018-08-16,CANDIDATE,0.000,0,0,...,-0.034,0.070,0.042,0.072,0.002,0.071,-0.027,0.074,0.027,0.074
3,10848459,K00754.01,NaN,FALSE POSITIVE,Done,2018-08-16,FALSE POSITIVE,0.000,0,1,...,0.147,0.078,0.289,0.079,-0.257,0.072,0.099,0.077,0.276,0.076
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,0,...,-0.090,0.180,0.100,0.140,0.070,0.180,0.020,0.160,0.070,0.200


In [6]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", module="lightgbm")

In [7]:
df_kp = df[['koi_pdisposition',
'koi_period', 'koi_duration', 'koi_depth', 'koi_impact', 'koi_ror',
               'koi_srho', 'koi_prad', 'koi_teq',
               'koi_insol', 'koi_smet',
               'koi_model_snr', 'koi_num_transits']]

In [8]:
le_disp = LabelEncoder()
df_kp['koi_pdisposition'] = le_disp.fit_transform(df_kp['koi_pdisposition'])

/tmp/ipython-input-1011288476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kp['koi_pdisposition'] = le_disp.fit_transform(df_kp['koi_pdisposition'])


In [9]:
imputer = KNNImputer(n_neighbors=5)
X = imputer.fit_transform(df_kp.drop('koi_pdisposition', axis=1))
y = df_kp['koi_pdisposition'].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
model = XGBClassifier(random_state=42)
params = model.get_params()
params

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'feature_weights': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

# OPTUNA FOR XGBoostClassifier

In [21]:
def objective(trial):
    # Search space for hyperparameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'random_state': 42
    }

    model = XGBClassifier(**params)

    # Use cross-validation score as objective
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()

    return score

In [22]:
# Create study and optimize
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100)

[I 2026-01-12 12:31:34,219] A new study created in memory with name: no-name-147e55fe-b742-4c05-9221-dfa7e41ca935
[I 2026-01-12 12:31:38,304] Trial 0 finished with value: 0.8327611233171396 and parameters: {'n_estimators': 300, 'learning_rate': 0.014728174923723025, 'max_depth': 6, 'subsample': 0.6405222106967778, 'colsample_bytree': 0.8906325468342102, 'min_child_weight': 2, 'gamma': 0.32723073612240894, 'reg_alpha': 0.6834279950242145, 'reg_lambda': 0.019946022856996315}. Best is trial 0 with value: 0.8327611233171396.
[I 2026-01-12 12:31:42,508] Trial 1 finished with value: 0.8253082983603471 and parameters: {'n_estimators': 400, 'learning_rate': 0.2435466686363645, 'max_depth': 8, 'subsample': 0.8938662547646061, 'colsample_bytree': 0.7481889400814968, 'min_child_weight': 5, 'gamma': 0.01608421702444629, 'reg_alpha': 0.9977493285013646, 'reg_lambda': 0.2561976015731108}. Best is trial 0 with value: 0.8327611233171396.
[I 2026-01-12 12:31:47,416] Trial 2 finished with value: 0.82262

In [23]:
print("Best parameters:", study.best_params)
print("Best F1 score:", study.best_value)

Best parameters: {'n_estimators': 400, 'learning_rate': 0.015603483038026065, 'max_depth': 6, 'subsample': 0.7299759595102386, 'colsample_bytree': 0.8377273956982814, 'min_child_weight': 6, 'gamma': 0.22255269708412592, 'reg_alpha': 0.424023705682538, 'reg_lambda': 0.9913150951095967}
Best F1 score: 0.8378268576982426


In [34]:
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [35]:
plot_optimization_history(study).show()

# OPTUNA FOR CatBoost

In [24]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1500, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),  # For imbalanced data
        'verbose': 0,
        'random_seed': 42
    }

    model = CatBoostClassifier(**params)

    # Cross-validation
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()

    return score

In [25]:

# Optimize
study_cat = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_cat.optimize(objective_catboost, n_trials=100)

[I 2026-01-12 12:40:45,090] A new study created in memory with name: no-name-89829b33-aab3-4963-a5fb-36aef25d4129
[I 2026-01-12 12:40:54,847] Trial 0 finished with value: 0.8191755203719527 and parameters: {'iterations': 500, 'learning_rate': 0.2290330451106017, 'depth': 5, 'l2_leaf_reg': 3.4483172288065695, 'border_count': 151, 'bagging_temperature': 0.5502588463644896, 'random_strength': 8.389321435652018, 'scale_pos_weight': 8.359776968326882}. Best is trial 0 with value: 0.8191755203719527.
[I 2026-01-12 12:41:05,236] Trial 1 finished with value: 0.8220906890838346 and parameters: {'iterations': 700, 'learning_rate': 0.09224654425922221, 'depth': 4, 'l2_leaf_reg': 2.081317066515921, 'border_count': 99, 'bagging_temperature': 0.7513926014816692, 'random_strength': 9.749764114367725, 'scale_pos_weight': 4.338323655488214}. Best is trial 1 with value: 0.8220906890838346.
[I 2026-01-12 12:41:15,507] Trial 2 finished with value: 0.7246531883013352 and parameters: {'iterations': 600, 'le

KeyboardInterrupt: 

In [26]:
print("Best CatBoost parameters:", study_cat.best_params)
print("Best F1 score:", study_cat.best_value)

Best CatBoost parameters: {'iterations': 400, 'learning_rate': 0.04216598764315635, 'depth': 6, 'l2_leaf_reg': 3.663779798834057, 'border_count': 116, 'bagging_temperature': 0.17142116748408584, 'random_strength': 4.070004945458448, 'scale_pos_weight': 1.4359614983167077}
Best F1 score: 0.8425166566572576


# OPTUNA FOR LightGBM

In [27]:

def objective_lightgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),  # For imbalanced data
        'boosting_type': 'gbdt',
        'verbose': -1,
        'random_state': 42
    }

    model = LGBMClassifier(**params)

    # Cross-validation
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()

    return score


In [28]:
# Optimize
study_lgb = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_lgb.optimize(objective_lightgbm, n_trials=100)

[I 2026-01-12 13:45:30,741] A new study created in memory with name: no-name-b4224e09-2277-4a4d-83db-5eb12fcede6e
[I 2026-01-12 13:45:35,068] Trial 0 finished with value: 0.7842077437307382 and parameters: {'n_estimators': 900, 'learning_rate': 0.0038086770739147197, 'max_depth': 3, 'num_leaves': 150, 'min_child_samples': 65, 'subsample': 0.9858510654468484, 'subsample_freq': 8, 'colsample_bytree': 0.7466678765367529, 'reg_alpha': 0.3398553900767147, 'reg_lambda': 0.6720344838160778, 'min_split_gain': 0.9559438713592047, 'scale_pos_weight': 3.7695807510945003}. Best is trial 0 with value: 0.7842077437307382.
[I 2026-01-12 13:45:46,212] Trial 1 finished with value: 0.8163159955708605 and parameters: {'n_estimators': 700, 'learning_rate': 0.011480617774176409, 'max_depth': 12, 'num_leaves': 86, 'min_child_samples': 51, 'subsample': 0.8265383419565231, 'subsample_freq': 5, 'colsample_bytree': 0.7168655194852233, 'reg_alpha': 0.6898590751349563, 'reg_lambda': 0.30105966486859315, 'min_spli

In [29]:
print("Best LightGBM parameters:", study_lgb.best_params)
print("Best F1 score:", study_lgb.best_value)

Best LightGBM parameters: {'n_estimators': 800, 'learning_rate': 0.04554947977677974, 'max_depth': 11, 'num_leaves': 63, 'min_child_samples': 26, 'subsample': 0.9706711709803809, 'subsample_freq': 8, 'colsample_bytree': 0.8753746188449917, 'reg_alpha': 0.8081540413430114, 'reg_lambda': 0.6361592585877186, 'min_split_gain': 0.7310816204125187, 'scale_pos_weight': 2.0382875855239857}
Best F1 score: 0.8416112991675021


# Random Forest

In [30]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective_randomforest(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'max_samples': trial.suggest_float('max_samples', 0.5, 1.0) if trial.params['bootstrap'] else None,
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.2),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample', None]),
        'random_state': 42,
        'n_jobs': -1  # Use all CPU cores
    }

    # Remove max_samples if bootstrap is False
    if not params['bootstrap']:
        params.pop('max_samples')

    model = RandomForestClassifier(**params)

    # Cross-validation
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()

    return score

In [31]:
# Optimize
study_rf = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_rf.optimize(objective_randomforest, n_trials=100)

[I 2026-01-12 14:28:53,548] A new study created in memory with name: no-name-b1406a3e-e255-46d9-80ca-25e927e9868a
[I 2026-01-12 14:29:19,266] Trial 0 finished with value: 0.36461609865797084 and parameters: {'n_estimators': 950, 'criterion': 'gini', 'max_depth': 25, 'min_samples_split': 10, 'min_samples_leaf': 8, 'max_features': 'sqrt', 'bootstrap': False, 'min_impurity_decrease': 0.12251060064911082, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.36461609865797084.
[I 2026-01-12 14:29:45,782] Trial 1 finished with value: 0.7470682318486814 and parameters: {'n_estimators': 200, 'criterion': 'entropy', 'max_depth': 15, 'min_samples_split': 14, 'min_samples_leaf': 2, 'max_features': None, 'bootstrap': True, 'max_samples': 0.7493302082647904, 'min_impurity_decrease': 0.06578208994643764, 'class_weight': None}. Best is trial 1 with value: 0.7470682318486814.
[I 2026-01-12 14:29:54,645] Trial 2 finished with value: 0.6471314543954648 and parameters: {'n_estimators': 20

KeyboardInterrupt: 

In [32]:
print("Best Random Forest parameters:", study_rf.best_params)
print("Best F1 score:", study_rf.best_value)

Best Random Forest parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 11, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': None, 'bootstrap': True, 'max_samples': 0.6588304820609029, 'min_impurity_decrease': 0.001220175704455228, 'class_weight': None}
Best F1 score: 0.8352985175589861
